# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
!pip install cartopy geoviews pyproj

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mata-utu,-13.2816,-176.1745,27.41,81,89,7.26,WF,1731266593
1,1,port-aux-francais,-49.3500,70.2167,4.44,83,62,9.97,TF,1731266594
2,2,guerrero negro,27.9769,-114.0611,23.59,31,92,1.38,MX,1731266476
3,3,lagunas,-5.2269,-75.6753,29.56,58,100,1.47,PE,1731266596
4,4,albany,42.6001,-73.9662,12.23,41,100,1.34,US,1731266540


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
Map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
Map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 40) & (city_data_df["Max Temp"] > 30) \
    & (city_data_df["Wind Speed"] < 10.0) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrow_city_df=narrow_city_df.dropna()

# Display sample data
narrow_city_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
188,188,obera,-27.4871,-55.1199,33.71,19,0,1.08,AR,1731266757
254,254,concordia,-31.3930,-58.0209,32.66,37,0,3.13,AR,1731266646
328,328,brisas de zicatela,15.8369,-97.0419,30.29,70,0,2.06,MX,1731266969
387,387,puerto rico,-26.7960,-55.0240,35.09,22,0,1.37,AR,1731267036
421,421,benjamin aceval,-24.9667,-57.5667,39.00,20,0,1.39,PY,1731267075
512,512,gobernador virasora,-28.0500,-56.0333,32.78,36,0,1.34,AR,1731267185


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity 
hotel=narrow_city_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel["Hotel Name"]=""

# Display sample data
hotel

/var/folders/9l/m439qlxd0dj0hplldndnm1dh0000gn/T/ipykernel_90415/1692646412.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel["Hotel Name"]=""


,City,Country,Lat,Lng,Humidity,Hotel Name
188,obera,AR,-27.4871,-55.1199,19,
254,concordia,AR,-31.3930,-58.0209,37,
328,brisas de zicatela,MX,15.8369,-97.0419,70,
387,puerto rico,AR,-26.7960,-55.0240,22,
421,benjamin aceval,PY,-24.9667,-57.5667,20,
512,gobernador virasora,AR,-28.0500,-56.0333,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey":geoapify_key,
          "limit":10
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel.loc[index, 'City']} - nearest hotel: {hotel.loc[index, 'Hotel Name']}")

# Display sample data
hotel

Starting hotel search
obera - nearest hotel: No hotel found
concordia - nearest hotel: D'Charruas
brisas de zicatela - nearest hotel: Casa de Olas
puerto rico - nearest hotel: Suizo
benjamin aceval - nearest hotel: No hotel found
gobernador virasora - nearest hotel: Arami


,City,Country,Lat,Lng,Humidity,Hotel Name
188,obera,AR,-27.4871,-55.1199,19,No hotel found
254,concordia,AR,-31.3930,-58.0209,37,D'Charruas
328,brisas de zicatela,MX,15.8369,-97.0419,70,Casa de Olas
387,puerto rico,AR,-26.7960,-55.0240,22,Suizo
421,benjamin aceval,PY,-24.9667,-57.5667,20,No hotel found
512,gobernador virasora,AR,-28.0500,-56.0333,36,Arami


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
Map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols=["Hotel Name","Country"])

# Display the map
Map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)